In [ ]:
# Corners of the page in global frame
LEFT_TOP = [-0.0904, 0.4208, 0.22, 0, 0, 1]
LEFT_BOTTOM = [-0.0904, 0.2049, 0.217, 0, 0, 1]
RIGHT_BOTTOM = [0.1890, 0.2049, 0.217, 0, 0, 1]
RIGHT_TOP = [0.1890, 0.4208, 0.22, 0, 0, 1]

# Function to convert page coordinates to global
def paper_coordinates(x,y, up=True):
    cords = []
    if (x > 0.2794) or (y > 0.2159) or (x < 0) or (y < 0):
        raise RuntimeError("Your coordinates are off the paper")
    cords.append(LEFT_BOTTOM[0]+x)
    cords.append(LEFT_BOTTOM[1]+y)
    if up:
        cords.append(0.225)
    else:
        cords.append(0.219-(0.002*((0.2159-y)/0.2159)))
    
    return cords + [0, 0, 0]

# Example use: pen at (0.1,0.1) on the page with the pen up
point = paper_coordinates(0.1,0.1, up=True)

In [ ]:
import xml.etree.ElementTree as ET

svg_file = 'mona_lisa.svg'

# Create a list of coordinates from SVG 
def svg_polyline_to_coordinates(svg_file):
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # Namespace handling
    namespaces = {'svg': 'http://www.w3.org/2000/svg'}
    
    polylines = root.findall('.//svg:polyline', namespaces)
    all_coordinates = []

    max_x = float('-inf')
    max_y = float('-inf')

    for polyline in polylines:
        points = polyline.get('points')
        if points:
            # Split the points string into individual coordinate pairs
            points_list = points.strip().split(',')
            # Extract x and y coordinates by pairing adjacent items in the list
            coordinates = [(float(points_list[i]), float(points_list[i+1])) for i in range(0, len(points_list), 2)]
            all_coordinates.append(coordinates)

            # Update max x and y values
            for x, y in coordinates:
                if x > max_x:
                    max_x = x
                if y > max_y:
                    max_y = y
    
    return all_coordinates,(max_x,max_y)

coordinates,max_vals = svg_polyline_to_coordinates(svg_file)

In [ ]:
import MyUR3e
import rclpy
import time
from IPython.display import HTML

# Initiate ROS2 and MyUR3e Class
rclpy.init()
myrobot = MyUR3e.MyUR3e()

In [ ]:
# Comment this out once pen is closed in the gripper
# myrobot.move_gripper(0,100,100) # Open
# time.sleep(2)
# myrobot.move_gripper(255,100,100) # Close

In [ ]:
# Rotate image 90 degrees counter clockwise for portrait orientation
# Can comment out if not needed

cx, cy = max_vals[0] / 2, max_vals[1] / 2

max_x = float('-inf')
max_y = float('-inf')

new_coordinates = []
for line in coordinates:
    rotated_points = []
    for x, y in line:
        # Translate to origin
        translated_x = x - cx
        translated_y = y - cy
    
        # Rotate
        rotated_x = -translated_y
        rotated_y = translated_x
    
        # Translate back
        final_x = rotated_x + cx
        final_y = rotated_y + cy

        # Find new max values
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
    
        rotated_points.append((final_x, final_y))
    new_coordinates.append(rotated_points)

coordinates = new_coordinates

In [ ]:
# height of paper is 0.2159m
# width of paper is 0.2794m
height = 0.2
scale = height / max_vals[1]
offset_x = (0.2159 - max_val[0]*scale) / 2
offset_y = (0.2794 - max_val[1]*scale) / 2

sim_trajectory = [] # collection of all trajectories for plot
for line in coordinates:
    drawing_trajectory = []
    for i,position in enumerate(line):
        x = position[0]*scale + offset_x
        y = (0.2159-(position[1]*scale+ offset_y))
        
        if i == 0:
            drawing_trajectory.append(paper_coordinates(x,y,up=True)) # pen up at first point
            drawing_trajectory.append(paper_coordinates(x,y,up=False)) # pen down at first point
        else:
            drawing_trajectory.append(paper_coordinates(x,y,up=False)) # interpolated points
    drawing_trajectory.append(paper_coordinates(x,y,up=True)) # pen up at last point
    sim_trajectory.extend(drawing_trajectory)

    # This line actually moves the robot
    myrobot.move_global(drawing_trajectory,time_step=(.4,0.06),sim=False)

In [ ]:
myrobot.clear_sim()
myrobot.move_global(sim_trajectory,time_step=(.6,0.1),sim=True)
HTML(filename="ur3e_trajectory.html")